programa skirta perskaičiuoti realų iš ESO gautą elektros gaminimą / suvartojimą į netbillingo modelį

In [3]:
import pandas as pd
from datetime import timedelta, date

# importuojame iš ESO valandinius duomenis ir juos paruošiame analizei
# importuojamas grynai žalias CVS failas iš ESO
# rezultatas - df1 datasetas

#
#
# SVARBU:
# nustatome datas tarp kuriu norime atlikti analize
start_date = pd.to_datetime(date(2021, 11, 1))
end_date = pd.to_datetime(date(2022, 11, 1))

import pandas as pd

df0 = pd.read_csv("Valandiniai [2021-11 - 2022-11].csv", header =0, sep=";", usecols=[12,13,14])
df_plus = df0[df0["Energijos tipas"]=='P+']
df_minus = df0[df0["Energijos tipas"]=='P-']
df1=pd.merge(df_plus, df_minus, on="Data, valanda")
df1.drop(['Energijos tipas_x', 'Energijos tipas_y'], axis=1, inplace=True)
df1.rename(columns={"Data, valanda":"datetime", "Kiekis, kWh_x":"kwh+", "Kiekis, kWh_y":"kwh-"}, inplace=True)
df1[['date', 'hrs']] = df1["datetime"].str.split(expand=True)
df1[['hh', 'mm', 'ss']] = df1['hrs'].str.split(':', expand=True)
df1.drop(['hrs', 'mm', 'ss'], axis=1, inplace=True)
df1['hh'] = df1['hh'].astype('int')
df1['kwh+'] = df1['kwh+'].str.replace(',','.')
df1['kwh-'] = df1['kwh-'].str.replace(',','.')
df1['kwh+'] = df1['kwh+'].astype('float')
df1['kwh-'] = df1['kwh-'].astype('float')
df1["ddtt"]= pd.to_datetime(df1['datetime'], infer_datetime_format=True)
df1.drop('datetime', axis=1, inplace=True)

# atrenkame duomenis kurie yra tarp datų
import time
df2 = df1[(df1['ddtt']>=start_date)&(df1['ddtt']<end_date)]


In [4]:
# nuskaitome NordPool kainas
# kwh+ energija gauta iš tinko
# kwh- energija pateikta į tinklą

import requests
nordpool_pricing = pd.DataFrame(columns = ['datetime', 'ddtt', 'NP'])
r = []

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

print ("gaunamos Nordpool kainos, progresas: ")
count = 0

for single_date in daterange(start_date, end_date):
    wd = single_date.isoweekday()
    d_date = single_date.strftime("%Y-%m-%d")
    count +=1
    if count >= 30: 
      print(".", end = "\n")
      count = 0
    else: print (".", end = ' ')
     

    
    l_date = d_date.split('-')
    dls = "https://np-lt-day-ahead-pricing.azurewebsites.net/api/date/" + d_date + "?mode=np"
    r = requests.get(dls)
    d = r.json()
    d.pop('metadata')
    d1 = {}
    
    for k,m in d.items():
      k = d_date + " " + k + ":00:00"
      nordpool_pricing = nordpool_pricing.append({'datetime': k, 'NP': m}, ignore_index=True, sort=False)
nordpool_pricing["ddtt"]= pd.to_datetime(nordpool_pricing['datetime'], infer_datetime_format=True)
nordpool_pricing.drop(['datetime'], axis=1, inplace=True)
df2=pd.merge(df2, nordpool_pricing, on="ddtt")    
df2


gaunamos Nordpool kainos, progresas: 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . 

,kwh+,kwh-,date,hh,ddtt,NP
0,1.7919,0.0,2021.11.10,0,2021-11-10 00:00:00,1.864
1,1.7278,0.0,2021.11.10,1,2021-11-10 01:00:00,8.052
2,1.7351,0.0,2021.11.10,2,2021-11-10 02:00:00,7.736
3,1.4527,0.0,2021.11.10,3,2021-11-10 03:00:00,7.760
4,0.5604,0.0,2021.11.10,4,2021-11-10 04:00:00,7.831
...,...,...,...,...,...,...
8417,0.7909,0.0,2022.10.31,19,2022-10-31 19:00:00,39.043
8418,0.8585,0.0,2022.10.31,20,2022-10-31 20:00:00,41.532
8419,0.7752,0.0,2022.10.31,21,2022-10-31 21:00:00,28.671
8420,1.0263,0.0,2022.10.31,22,2022-10-31 22:00:00,27.210


In [5]:
# padauginam kwh is NP kainu - nustatom kiek pajamų / išlaidų NP kainomis būtų buvę
# paverčiam kainas į EUR (iš centų)

# kwh+ energija gauta iš tinko
# kwh- energija pateikta į tinklą

df2['kwh+ kaina'] = df2['kwh+'] * df2['NP'] / 100 
df2['kwh- kaina'] = df2['kwh-'] * df2['NP'] / 100
df2

,kwh+,kwh-,date,hh,ddtt,NP,kwh+ kaina,kwh- kaina
0,1.7919,0.0,2021.11.10,0,2021-11-10 00:00:00,1.864,0.033401,0.0
1,1.7278,0.0,2021.11.10,1,2021-11-10 01:00:00,8.052,0.139122,0.0
2,1.7351,0.0,2021.11.10,2,2021-11-10 02:00:00,7.736,0.134227,0.0
3,1.4527,0.0,2021.11.10,3,2021-11-10 03:00:00,7.760,0.112730,0.0
4,0.5604,0.0,2021.11.10,4,2021-11-10 04:00:00,7.831,0.043885,0.0
...,...,...,...,...,...,...,...,...
8417,0.7909,0.0,2022.10.31,19,2022-10-31 19:00:00,39.043,0.308791,0.0
8418,0.8585,0.0,2022.10.31,20,2022-10-31 20:00:00,41.532,0.356552,0.0
8419,0.7752,0.0,2022.10.31,21,2022-10-31 21:00:00,28.671,0.222258,0.0
8420,1.0263,0.0,2022.10.31,22,2022-10-31 22:00:00,27.210,0.279256,0.0


In [6]:
df3 = df2.groupby(pd.Grouper(key='ddtt', freq="M")).agg({'kwh+': 'sum', 'kwh+ kaina': 'sum',  'kwh-': 'sum','kwh- kaina': 'sum'}).round(2)
df3

,kwh+,kwh+ kaina,kwh-,kwh- kaina
ddtt,,,,
2021-11-30,807.02,124.02,5.88,1.00
2021-12-31,1511.83,336.73,2.11,0.45
2022-01-31,1488.37,219.55,3.80,0.59
2022-02-28,985.82,106.76,71.05,7.40
2022-03-31,518.07,84.63,460.88,71.64
2022-04-30,392.27,41.59,609.29,83.40
2022-05-31,207.21,35.32,966.74,172.39
2022-06-30,160.38,38.08,1191.69,267.01
2022-07-31,211.77,68.15,1088.85,337.22


In [7]:
print("kwh+ sunaudota iš tinklo energija")
print("kwh- pateikta į tinklą energija")
print()
df3['kwh+ vidurkis'] = df3['kwh+ kaina'] / df3['kwh+']
df3['kwh- vidurkis'] = df3['kwh- kaina'] / df3['kwh-']
df3

kwh+ sunaudota iš tinklo energija
kwh- pateikta į tinklą energija



,kwh+,kwh+ kaina,kwh-,kwh- kaina,kwh+ vidurkis,kwh- vidurkis
ddtt,,,,,,
2021-11-30,807.02,124.02,5.88,1.00,0.153676,0.170068
2021-12-31,1511.83,336.73,2.11,0.45,0.222730,0.213270
2022-01-31,1488.37,219.55,3.80,0.59,0.147510,0.155263
2022-02-28,985.82,106.76,71.05,7.40,0.108296,0.104152
2022-03-31,518.07,84.63,460.88,71.64,0.163356,0.155442
2022-04-30,392.27,41.59,609.29,83.40,0.106024,0.136881
2022-05-31,207.21,35.32,966.74,172.39,0.170455,0.178321
2022-06-30,160.38,38.08,1191.69,267.01,0.237436,0.224060
2022-07-31,211.77,68.15,1088.85,337.22,0.321811,0.309703
